In [21]:
! pip install pandas matplotlib torch cairosvg tqdm openai

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [ ]:
import os
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import torch
import cairosvg
import numpy as np
import io
import math
import sys
import base64
import json
import time
import csv
from tqdm import tqdm
from openai import OpenAI

# ---- CONFIGURATION ----
MODEL_NAME = "gpt-4o"   # Vision-language model
# Initializing OpenAI client - see https://platform.openai.com/docs/quickstart?context=python
OPENAI_KEY = "<OPENAI_KEY>"
client = OpenAI(api_key=OPENAI_KEY)

CUSTOM_PROMPT = """
You are an AI Agent with specilaised knowledge in reading and understanding map data. Analyze the following map and using information from the steps and examples given below, answer the question.

Steps to follow:

Identify Map-Related Elements in the Question
 Carefully understand the question to determine the key geographical features, locations, or spatial characteristics it refers to.


Locate the Identified Elements on the Map
 Find and observe these features or entities on the map provided. Pay attention to patterns, distributions, directions, scales, or any visual cues.


Apply Logical Reasoning
 Use spatial reasoning and contextual clues from the map to draw connections between the question and the observed map features.


Formulate a Concise Answer
 Based on your reasoning, arrive at a clear and accurate answer. Return only a word or phrase, as required—no explanation is needed. 
 If adequate data is not present, give answer as "no data". 
 If you have all the data and there is no answer to the question, give answer "none". If it is a counting problem, give answer 0. 
 If you have all the data and it is not possible to answer the question, give answer "not possible".

Assuming we are talking about a map with election results for USA. This  map contains  the voter breakdown across the United States, including the number of votes cast and the winning party in each state. Some examples of questions and there answers are as follows:


Question: Count the number states on the west coast where Democrats won.
Answer: 3


Questions: Based on the information given in the map, who won the election, Democrats or Republicans?
Answer: Democrats

Questions: Based on the information given in the map, if both Democrats and Republicans win 25 states each, do we have more blue states or red states?
Answer: neither

Question: List the top 4 states in terms of seats where the republicans won
Answer: Texas, Georgia, Missouri, Tenessee

Question: Rank these states in ascending order of seats - kansas, south carolina, nebraska, oklahoma, colorado, wisconsin
Answer: nebraska, kansas, oklahoma, south carolina, colorado, wisconsin

Question: Based on reasoning, Answer the following:
     Montana : Wyoming :: North Dakota : ?
Answer: South Dakota

Now, Answer the Question below based on the information, instruction and examples above:

"""
INPUT_CSV = "data/typed_questions.csv"
OUTPUT_CSV = f"results/{MODEL_NAME.split('/')[-1]}_results.csv"  
IMAGE_BASE_FOLDER = "data/imgs/"

# ---- HELPER FUNCTIONS ----
def find_image_recursive(base_folder: str, image_name: str) -> str | None:
    """Recursively find image by filename in base_folder. Returns full path or None."""
    for root, _, files in os.walk(base_folder):
        if image_name in files:
            return os.path.join(root, image_name)
    return None


def load_image_safely(image_path: str, output_format: str = "PNG"):
    """
    Load an image safely and return a base64-encoded data URI.
    Supports PNG, JPEG, GIF, WEBP, and SVG.
    """
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    except Exception as e:
        print(f"❌ Failed to load image {image_path}: {e}")
        return None

def ask_image_question(image_path: str, question: str, custom_prompt: str, index:int) -> str:
    image = load_image_safely(image_path)
    if image is None:
        return "Error loading image"
    
    # ✅ OpenAI requires chat template
    task = {
        "custom_id": f"task-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            # This is what you would have in your Chat Completions API call
            "model": MODEL_NAME,
            "temperature": 0.0,
            "max_tokens": 128,
            "messages": [
                {
                    "role": "system",
                    "content": custom_prompt
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": question
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{image}"
                            }
                        },
                    ],
                }
            ]            
        }
    }
    
    return task

# Load already answered pairs from output CSV to a set for quick lookup
def load_already_answered(output_csv):
    answered_set = set()
    try:
        df_out = pd.read_csv(output_csv)
        for _, row in df_out.iterrows():
            answered_set.add( (row['image_name'], row['question']) )
    except FileNotFoundError:
        # Output file does not exist yet
        pass
    return answered_set

In [23]:
def process_csv(input_csv, output_csv, base_folder, custom_prompt):
    answered_pairs = load_already_answered(output_csv)
    df_all = pd.read_csv(input_csv)
    batch_size = 100
    n_batches = math.ceil(len(df_all) / batch_size)
    start_idx = 0 
    j = start_idx
    for i in range(n_batches):
        tasks = []
        index_to_meta = {}

        # Collect until we have 100 valid tasks or we exhaust df_all
        while len(tasks) < batch_size and j < len(df_all):
            row = df_all.iloc[j]
            image_name, question = row['image_name'], row['question']
            j += 1  # move pointer no matter what

            # Skip if already answered
            if (image_name, question) in answered_pairs:
                continue

            image_path = find_image_recursive(base_folder, image_name)

            try:
                if not image_path:
                    # Directly save "Image not found"
                    with open(output_csv, mode='a', newline='', encoding='utf-8') as f_out:
                        writer = csv.DictWriter(f_out, fieldnames=["image_name", "question", "llm_answer"])
                        if f_out.tell() == 0:
                            writer.writeheader()
                        writer.writerow({
                            "image_name": image_name,
                            "question": question,
                            "llm_answer": "Image not found"
                        })
                else:
                    task = ask_image_question(image_path, question, custom_prompt, j)
                    tasks.append(task)
                    index_to_meta[f"task-{j}"] = (image_name, question)

            except Exception as e:
                print(f"❌ Error processing {image_name} - {question}: {e}")
                
        # If no tasks left, stop early
        if not tasks:
            print("✅ No new tasks to run.")
            continue

        # Creating the file
        file_name = f"openai_results/batch_tasks_mapqa_{i}.jsonl"
        with open(file_name, 'w') as file:
            for obj in tasks:
                file.write(json.dumps(obj) + '\n')
        # Uploading the file 
        batch_file = client.files.create(
            file=open(file_name, "rb"),
            purpose="batch"
        )
        
        print(i, j, batch_file.id)
        
        # Creating the job
        batch_job = client.batches.create(
            input_file_id=batch_file.id,
            endpoint="/v1/chat/completions",
            completion_window="24h"
        )
        time.sleep(10)

In [24]:
# process_csv(INPUT_CSV, OUTPUT_CSV, IMAGE_BASE_FOLDER, CUSTOM_PROMPT)

In [35]:
import io
import json
import pandas as pd
from tqdm import tqdm

def submit_batches(input_csv, base_folder, custom_prompt, batch_metadata_file="openai_results/batch_jobs_metadata_4o.json"):
    """
    Submit batch jobs in chunks of 100 while avoiding duplication.
    Already submitted chunks are skipped.
    """
    # Load existing metadata or start fresh
    try:
        with open(batch_metadata_file, "r") as f:
            batch_metadata = json.load(f)
    except FileNotFoundError:
        batch_metadata = []

    submitted_indices = {meta["start_index"] for meta in batch_metadata}

    # Read input CSV in chunks
    for chunk_start, chunk in enumerate(pd.read_csv(input_csv, chunksize=100)):
        start_index = chunk_start * 100
        if start_index in submitted_indices:
            print(f"⏩ Skipping chunk starting at index {start_index}, already submitted")
            continue

        df = chunk
        tasks = []
        index_to_meta = {}

        for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Preparing tasks for chunk starting at {start_index}"):
            image_name, question = row['image_name'], row['question']
            image_path = find_image_recursive(base_folder, image_name)

            if not image_path:
                print(f"⚠️ Image not found: {image_name}")
                continue

            task = ask_image_question(image_path, question, custom_prompt, idx)
            tasks.append(task)
            index_to_meta[f"task-{idx}"] = (image_name, question)

        if not tasks:
            print(f"✅ No valid tasks for chunk starting at {start_index}")
            continue

        # Convert tasks to bytes for upload
        jsonl_bytes = "\n".join(json.dumps(task) for task in tasks).encode("utf-8")
        file_like = io.BytesIO(jsonl_bytes)

        # Upload batch file
        batch_file = client.files.create(file=file_like, purpose="batch")
        batch_job = client.batches.create(
            input_file_id=batch_file.id,
            endpoint="/v1/chat/completions",
            completion_window="24h"
        )

        # Save metadata
        batch_metadata.append({
            "batch_id": batch_job.id,
            "start_index": start_index,
            "size": len(df),
            "status": "submitted",
            "index_to_meta": index_to_meta
        })

        with open(batch_metadata_file, "w") as f:
            json.dump(batch_metadata, f, indent=4)

        print(f"🟢 Submitted batch {batch_job.id} for chunk starting at {start_index}")


In [36]:
submit_batches(INPUT_CSV, IMAGE_BASE_FOLDER, CUSTOM_PROMPT)

⏩ Skipping chunk starting at index 0, already submitted
⏩ Skipping chunk starting at index 100, already submitted
⏩ Skipping chunk starting at index 200, already submitted
⏩ Skipping chunk starting at index 300, already submitted
⏩ Skipping chunk starting at index 400, already submitted
⏩ Skipping chunk starting at index 500, already submitted
⏩ Skipping chunk starting at index 600, already submitted
⏩ Skipping chunk starting at index 700, already submitted
⏩ Skipping chunk starting at index 800, already submitted
⏩ Skipping chunk starting at index 900, already submitted
⏩ Skipping chunk starting at index 1000, already submitted
⏩ Skipping chunk starting at index 1100, already submitted
⏩ Skipping chunk starting at index 1200, already submitted
⏩ Skipping chunk starting at index 1300, already submitted
⏩ Skipping chunk starting at index 1400, already submitted
⏩ Skipping chunk starting at index 1500, already submitted
⏩ Skipping chunk starting at index 1600, already submitted
⏩ Skippin

In [37]:
def poll_and_save_results(output_csv, batch_metadata_file="openai_results/batch_jobs_metadata_4o.json"):
    with open(batch_metadata_file, "r") as f:
        batch_metadata = json.load(f)

    for meta in batch_metadata:
        batch_id = meta["batch_id"]
        start_index = meta["start_index"]
        index_to_meta = meta["index_to_meta"]

        if meta.get("status") == "completed":
            continue  # already processed

        batch_job = client.batches.retrieve(batch_id)
        status = batch_job.status
        print(f"⏳ Batch {batch_id} status: {status}")

        if status != "completed":
            print(f"❌ Batch {batch_id} not completed yet")
            continue

        # Download output file
        output_file = client.files.content(batch_job.output_file_id)
        lines = output_file.text.splitlines()

        results = []
        for line in lines:
            data = json.loads(line)
            try:
                content = data["response"]["body"]["choices"][0]["message"]["content"]
                if isinstance(content, list):
                    text = "".join([c.get("text", "") for c in content if c.get("type") == "text"])
                else:
                    text = content
                results.append(text)
            except Exception as e:
                print("Skipping malformed line:", e)

        # Write results mapped back to original CSV
        with open(output_csv, mode="a", newline="", encoding="utf-8") as f_out:
            writer = csv.DictWriter(f_out, fieldnames=["image_name", "question", "llm_answer"])
            if f_out.tell() == 0:
                writer.writeheader()

            # Map results using index_to_meta
            for idx, answer in enumerate(results):
                task_key = f"task-{start_index + idx}"
                image_name, question = index_to_meta.get(task_key, ("UNKNOWN", "UNKNOWN"))
                writer.writerow({
                    "image_name": image_name,
                    "question": question,
                    "llm_answer": answer if answer else "❌ No answer returned"
                })

        # Update metadata status
        meta["status"] = "completed"
        with open(batch_metadata_file, "w") as f:
            json.dump(batch_metadata, f, indent=4)

        print(f"✅ Results from batch {batch_id} saved to {output_csv}")


In [38]:
poll_and_save_results(OUTPUT_CSV)

⏳ Batch batch_68cd25b91fc8819096aee7d769906883 status: failed
❌ Batch batch_68cd25b91fc8819096aee7d769906883 not completed yet
⏳ Batch batch_68cd2606dd9c8190aabfe0c346c75af4 status: failed
❌ Batch batch_68cd2606dd9c8190aabfe0c346c75af4 not completed yet
⏳ Batch batch_68cd319adb1c8190a4da0b6fe864efa0 status: failed
❌ Batch batch_68cd319adb1c8190a4da0b6fe864efa0 not completed yet


In [39]:
import io
import json
import pandas as pd

def rerun_failed_batches(input_csv, base_folder, custom_prompt, batch_metadata_file="openai_results/batch_jobs_metadata.json"):
    """
    Rerun batches that failed using the saved metadata.
    """
    # Load existing metadata
    try:
        with open(batch_metadata_file, "r") as f:
            batch_metadata = json.load(f)
    except FileNotFoundError:
        print("❌ Metadata file not found")
        return

    rerun_metadata = []
    for meta in batch_metadata:
        if meta.get("status") == "failed":
            start_index = meta["start_index"]
            size = meta["size"]

            print(f"🔄 Rerunning failed batch {meta['batch_id']} (chunk {start_index}–{start_index+size-1})")

            # Load the corresponding CSV slice
            df = pd.read_csv(input_csv, skiprows=range(1, start_index+1), nrows=size)

            tasks = []
            index_to_meta = {}
            for idx, row in df.iterrows():
                image_name, question = row['image_name'], row['question']
                image_path = find_image_recursive(base_folder, image_name)

                if not image_path:
                    continue

                task = ask_image_question(image_path, question, custom_prompt, idx)
                tasks.append(task)
                index_to_meta[f"task-{idx}"] = (image_name, question)

            if not tasks:
                print(f"⚠️ No tasks to rerun for chunk starting at {start_index}")
                continue

            # Convert to JSONL and upload
            jsonl_bytes = "\n".join(json.dumps(task) for task in tasks).encode("utf-8")
            file_like = io.BytesIO(jsonl_bytes)

            batch_file = client.files.create(file=file_like, purpose="batch")
            batch_job = client.batches.create(
                input_file_id=batch_file.id,
                endpoint="/v1/chat/completions",
                completion_window="24h"
            )

            # Update metadata
            meta.update({
                "rerun_batch_id": batch_job.id,
                "status": "resubmitted",
                "index_to_meta": index_to_meta
            })
            rerun_metadata.append(meta)

            print(f"🟢 Resubmitted as batch {batch_job.id}")

    # Save updated metadata
    if rerun_metadata:
        with open(batch_metadata_file, "w") as f:
            json.dump(batch_metadata, f, indent=4)


In [40]:
def rerun_incomplete_batches(
    input_csv,
    base_folder,
    custom_prompt,
    batch_metadata_file="openai_results/batch_jobs_metadata.json"
):
    try:
        with open(batch_metadata_file, "r") as f:
            batch_metadata = json.load(f)
    except FileNotFoundError:
        print("❌ No batch metadata file found.")
        return

    for meta in batch_metadata:
        if meta.get("status") == "completed":
            continue  # skip finished ones

        start_index = meta["start_index"]
        size = meta["size"]

        print(f"🔄 Rerunning batch {meta['batch_id']} (status={meta['status']}) covering rows {start_index}–{start_index+size-1}")

        # reload chunk from CSV
        df = pd.read_csv(input_csv, skiprows=range(1, start_index+1), nrows=size)

        tasks = []
        index_to_meta = {}
        for idx, row in df.iterrows():
            image_name, question = row['image_name'], row['question']
            image_path = find_image_recursive(base_folder, image_name)
            if not image_path:
                continue
            task = ask_image_question(image_path, question, custom_prompt, idx)
            tasks.append(task)
            index_to_meta[f"task-{idx}"] = (image_name, question)

        if not tasks:
            print("⚠️ No tasks found for rerun.")
            continue

        # upload new file + create new batch
        buf = io.BytesIO()
        for obj in tasks:
            buf.write((json.dumps(obj) + "\n").encode("utf-8"))
        buf.seek(0)

        batch_file = client.files.create(file=buf, purpose="batch")
        batch_job = client.batches.create(input_file_id=batch_file.id, endpoint="/v1/chat/completions", completion_window="24h")

        # update metadata
        meta.update({
            "batch_id": batch_job.id,
            "status": "submitted",
            "index_to_meta": index_to_meta,
        })

    with open(batch_metadata_file, "w") as f:
        json.dump(batch_metadata, f, indent=4)

    print("✅ Incomplete batches resubmitted")


In [41]:
rerun_incomplete_batches(INPUT_CSV, IMAGE_BASE_FOLDER, CUSTOM_PROMPT)

🔄 Rerunning batch batch_68cc319d7c008190b11e31b6e875368a (status=failed) covering rows 2300–2399
🔄 Rerunning batch batch_68cc31dcbad4819093a44e3ae8989c6b (status=failed) covering rows 2400–2499
🔄 Rerunning batch batch_68cc3226c3ec819089ca9cc25c7f691a (status=failed) covering rows 8100–8199
✅ Incomplete batches resubmitted


In [19]:
import json
import time

def poll_and_update_batches(batch_metadata_file="openai_results/batch_jobs_metadata.json", poll_interval=30):
    """
    Poll all submitted batch jobs and update their status in metadata.
    Saves updated metadata after every cycle.
    """
    try:
        with open(batch_metadata_file, "r") as f:
            batch_metadata = json.load(f)
    except FileNotFoundError:
        print("❌ No batch metadata file found.")
        return

    all_done = False
    while not all_done:
        all_done = True
        for meta in batch_metadata:
            batch_id = meta["batch_id"]
            status = meta.get("status", "unknown")

            # Skip completed or failed ones if you only want live polling
            if status in ["completed", "failed", "expired", "canceled"]:
                continue

            try:
                job = client.batches.retrieve(batch_id)
                new_status = job.status
                if new_status != status:
                    print(f"🔄 Batch {batch_id} status updated: {status} → {new_status}")
                    meta["status"] = new_status
                    # also store finish_time if available
                    if hasattr(job, "completed_at") and job.completed_at:
                        meta["completed_at"] = job.completed_at
            except Exception as e:
                print(f"⚠️ Could not retrieve batch {batch_id}: {e}")

        # save metadata after each poll cycle
        with open(batch_metadata_file, "w") as f:
            json.dump(batch_metadata, f, indent=4)

        # check if all jobs are in terminal states
        if any(m["status"] not in ["completed", "failed", "expired", "canceled"] for m in batch_metadata):
            all_done = False
            print(f"⏳ Waiting {poll_interval}s before next check...")
            time.sleep(poll_interval)

    print("✅ All batches reached a terminal state.")


In [20]:
poll_and_update_batches()

🔄 Batch batch_68cc319d7c008190b11e31b6e875368a status updated: submitted → failed
🔄 Batch batch_68cc31dcbad4819093a44e3ae8989c6b status updated: submitted → failed
🔄 Batch batch_68cc3226c3ec819089ca9cc25c7f691a status updated: submitted → validating
⏳ Waiting 30s before next check...
🔄 Batch batch_68cc3226c3ec819089ca9cc25c7f691a status updated: validating → failed
✅ All batches reached a terminal state.
